In [3]:
ps = [('Pepe','M',32),
     ('Veronica','F',37),
     ('Juan','M',28)]

In [4]:
ps

[('Pepe', 'M', 32), ('Veronica', 'F', 37), ('Juan', 'M', 28)]

In [5]:
rdd = sc.parallelize(ps)

In [6]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423

Hasta que no va a hacer algo con los datos, no los lee. Por eso, aparece como un error.

In [7]:
rdd.collect()

[('Pepe', 'M', 32), ('Veronica', 'F', 37), ('Juan', 'M', 28)]

In [10]:
# si queremos calcular la media de toda la lista, primero seleccionamos el tercer índice de cada elemento (la edad)
[x[2] for x in ps]

[32, 37, 28]

In [11]:
import math

In [12]:
# calculamos la suma ahora:
sum([x[2] for x in ps])

97

In [13]:
# no podemos hacerlo sobre rdd porque los datos están distribuidos en varios nódulos
[x[2] for x in rdd]

TypeError: 'RDD' object is not iterable

In [14]:
# el equivalente a [x[2] for x in rdd] para usar rdd sería usar map
rdd.map(lambda x: x[2])

PythonRDD[1] at RDD at PythonRDD.scala:43

In [16]:
rdd.map(lambda x: x[2]).collect()

[32, 37, 28]

In [17]:
rdd.map(lambda x: x[2]).mean()

32.333333333333336

In [25]:
# vamos a calcularlo por género
kv = rdd.map(lambda x: (x[1],x[2]))
# En Python los índices van entre corchetes y las tuplas se ponen entre ()

In [26]:
kv.collect()

[('M', 32), ('F', 37), ('M', 28)]

In [22]:
# se pueden meter varios elementos en la clave: una tupla de tuplas:
kv2 = rdd.map(lambda x: ((x[1],x[0]),x[2]))

In [24]:
kv2.collect()
# la clae ahora está compuesta por sexo y nombre

[(('M', 'Pepe'), 32), (('F', 'Veronica'), 37), (('M', 'Juan'), 28)]

In [27]:
kv.groupByKey().collect()

[('M', <pyspark.resultiterable.ResultIterable at 0x7fa5ef6a1a10>),
 ('F', <pyspark.resultiterable.ResultIterable at 0x7fa5ef6a1dd0>)]

In [28]:
# para calcular las medias tenemos que transformar cada uno de los dos grupos que tenemos en valores
# para ello tenemos que hacer un map
kv.groupByKey().map(lambda x: sum(x[1])).collect()
# cada una de las x van a ser una tupla donde el primer elemento es la clave y el segundo elemento es el valor asociad
# es decir, cada x son cada una de las líneas de la salida anterior.

[60, 37]

In [29]:
kv.groupByKey().map(lambda x: (x[0],sum(x[1]))).collect()

[('M', 60), ('F', 37)]

In [31]:
# otra forma es con mapValues que coge directamente los valores (en vez de coger la clave y el valor, coge sólo el 
# valor) y simplifica el código.
kv.groupByKey().mapValues(lambda vs: sum(vs)).collect()

[('M', 60), ('F', 37)]

In [32]:
#calculamos la media: suma/longitud
kv.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).collect()

[('M', 30), ('F', 37)]